# Chapter 17: Representation Learning dan Generative Learning
## Autoencoders & Generative Adversarial Networks (GANs)

Bab ini membahas teknik pembelajaran tanpa pengawasan (*unsupervised*) untuk mempelajari representasi data yang padat. Kita akan mempelajari bagaimana **Autoencoders** mengompresi informasi dan bagaimana **GANs** menghasilkan data baru melalui kompetisi antar jaringan.

### Fokus Pembelajaran:
1. **Prinsip Dasar Autoencoder**: Efek *bottleneck*.
2. **Stacked & Convolutional Autoencoders**: Menangani data gambar secara hierarkis.
3. **Denoising & Sparse Autoencoders**: Teknik regularisasi representasi.
4. **Variational Autoencoders (VAE)**: Pintu masuk ke dunia model generatif.
5. **Generative Adversarial Networks (GAN)**: Dinamika Generator vs Discriminator.
6. **Deep Convolutional GAN (DCGAN)**: Standar untuk pembuatan gambar sintetis.

## 1. Undercomplete Autoencoders

Autoencoder yang paling sederhana adalah yang memiliki lapisan tersembunyi dengan dimensi lebih kecil dari input. Ini memaksa jaringan untuk mempelajari fitur yang paling penting (mirip dengan PCA jika lapisannya linear).

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Membuat data sintetis 3D yang sebenarnya berada pada bidang 2D
def generate_3d_data(m, w1=0.1, w2=0.3, noise=0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(100)
X_train = X_train - X_train.mean(axis=0)

# Membangun Autoencoder Linear (2 neuron di bottleneck)
encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape=[2])])
autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=0.1))
history = autoencoder.fit(X_train, X_train, epochs=20, verbose=0)

print("Pelatihan Autoencoder Linear selesai. Loss akhir:", history.history['loss'][-1])

## 2. Stacked Autoencoders

Stacked Autoencoder memiliki beberapa lapisan tersembunyi. Keuntungannya adalah kemampuan untuk mempelajari fitur yang lebih kompleks. Namun, jika terlalu banyak kapasitas, autoencoder bisa saja hanya "menyalin" tanpa belajar (overfitting).

In [ ]:
# Memuat dataset Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]

def build_stacked_autoencoder():
    # Encoder: 784 -> 100 -> 30
    encoder = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.Dense(30, activation="selu"),
    ])
    # Decoder: 30 -> 100 -> 784
    decoder = keras.models.Sequential([
        keras.layers.Dense(100, activation="selu", input_shape=[30]),
        keras.layers.Dense(28 * 28, activation="sigmoid"),
        keras.layers.Reshape([28, 28])
    ])
    return keras.models.Sequential([encoder, decoder])

stacked_ae = build_stacked_autoencoder()
stacked_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1.5))
print("Arsitektur Stacked Autoencoder siap.")

## 3. Convolutional Autoencoders

Sama seperti CNN untuk klasifikasi, Convolutional Autoencoder menggunakan lapisan `Conv2D` untuk encoder dan `Conv2DTranspose` untuk decoder (untuk menaikkan resolusi gambar kembali).

In [ ]:
conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    keras.layers.Conv2D(16, kernel_size=3, padding="SAME", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(32, kernel_size=3, padding="SAME", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(64, kernel_size=3, padding="SAME", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2)
])

conv_decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding="VALID", activation="selu", input_shape=[3, 3, 64]),
    keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding="SAME", activation="selu"),
    keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding="SAME", activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()

## 4. Variational Autoencoders (VAE)

VAE adalah model generatif. Alih-alih memetakan input ke titik tunggal di *latent space*, VAE memetakan input ke distribusi probabilitas. Ini memungkinkan kita menghasilkan gambar baru dengan melakukan sampling dari distribusi tersebut.

In [ ]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

# Encoder VAE
latent_dim = 2
inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation="selu")(z)
z = keras.layers.Dense(100, activation="selu")(z)
codings_mean = keras.layers.Dense(latent_dim)(z)
codings_log_var = keras.layers.Dense(latent_dim)(z)
codings = Sampling()([codings_mean, codings_log_var])
vae_encoder = keras.models.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

# Decoder VAE
decoder_inputs = keras.layers.Input(shape=[latent_dim])
x = keras.layers.Dense(100, activation="selu")(decoder_inputs)
x = keras.layers.Dense(150, activation="selu")(x)
x = keras.layers.Dense(28 * 28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
vae_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[outputs])

print("Variational Autoencoder berhasil didefinisikan.")

## 5. Generative Adversarial Networks (GANs)

GAN melatih dua model secara bersamaan:
- **Generator**: Membuat data palsu dari kebisingan acak (*random noise*).
- **Discriminator**: Menentukan apakah data itu asli (dari dataset) atau palsu (dari generator).

Proses ini adalah kompetisi terus-menerus hingga generator menghasilkan data yang tidak bisa dibedakan lagi oleh discriminator.

In [ ]:
# Generator GAN Sederhana
codings_size = 30
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

# Discriminator GAN Sederhana
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

gan = keras.models.Sequential([generator, discriminator])

# Penting: Discriminator harus dikompilasi secara terpisah
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False # Saat melatih GAN utuh, discriminator tidak dilatih
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

print("Struktur GAN dasar siap.")

## 6. Tantangan Melatih GAN

Melatih GAN jauh lebih sulit daripada CNN biasa. Beberapa tantangan utamanya:
1. **Nash Equilibrium**: Sulit menemukan keseimbangan antara dua pemain (Gen vs Disc).
2. **Mode Collapse**: Generator hanya menghasilkan satu variasi gambar yang sangat mirip (karena berhasil menipu discriminator).
3. **Ketidakstabilan**: Kehilangan (*loss*) discriminator bisa menjadi nol, sehingga generator tidak belajar apa-apa lagi.

## Kesimpulan Praktis

- **Autoencoders**: Sangat baik untuk ekstraksi fitur, pengurangan dimensi, dan deteksi anomali.
- **VAE**: Memungkinkan generasi data baru dengan kontrol pada *latent space*, namun gambar seringkali agak kabur.
- **GAN**: Menghasilkan gambar yang sangat tajam dan realistis, namun memerlukan teknik regularisasi dan stabilitas tingkat tinggi (seperti DCGAN).